# Imports de librerías y configuración de device

---



In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import string
import unicodedata

In [3]:
#device para entrenar el modelo
device="cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

In [4]:
#Unzip la data
!unzip /content/data_cc.zip -d /content

Archive:  /content/data_cc.zip
   creating: /content/data/
  inflating: /content/data/eng-fra.txt  
   creating: /content/data/names/
  inflating: /content/data/names/Arabic.txt  
  inflating: /content/data/names/Chinese.txt  
  inflating: /content/data/names/Czech.txt  
  inflating: /content/data/names/Dutch.txt  
  inflating: /content/data/names/English.txt  
  inflating: /content/data/names/French.txt  
  inflating: /content/data/names/German.txt  
  inflating: /content/data/names/Greek.txt  
  inflating: /content/data/names/Irish.txt  
  inflating: /content/data/names/Italian.txt  
  inflating: /content/data/names/Japanese.txt  
  inflating: /content/data/names/Korean.txt  
  inflating: /content/data/names/Polish.txt  
  inflating: /content/data/names/Portuguese.txt  
  inflating: /content/data/names/Russian.txt  
  inflating: /content/data/names/Scottish.txt  
  inflating: /content/data/names/Spanish.txt  
  inflating: /content/data/names/Vietnamese.txt  


# Preprocesamiento de los caracteres

## Preprocesamiento de las palabras

In [5]:
#Caracteres permitidos --> "_" representará a cualquier otra letra no conocida
chars_allowed=string.ascii_letters+" .,;'"+"_"

In [6]:
#Tamaño del vocabulario (servirá para determinar el tamaño de cada vector en OHE)
len_chars=len(chars_allowed)

In [7]:
#Función para transformar las palabras con signos distintos a caracteres conocidos
def oracion_to_ready(orac):
  sentence=""
  oracion_norm=unicodedata.normalize("NFD",orac)

  for char in oracion_norm:
    category=unicodedata.category(char)
    if category!="Mn":
      sentence+=char

  return sentence

## Convirtiendo a tensor

In [8]:
#Función que devuelve el índice para crear el vector OHE
def letter_index(letra):
  if letra not in chars_allowed:
    return chars_allowed.find("_")

  else:
    return chars_allowed.find(letra)

In [9]:
#Función que crea el vector OHE (el index representará dónde estará el 1 para la notación OHE)
def word_to_tensor(palabra):
  tensor_final=torch.zeros(size=(len(palabra),1,len_chars))

  palabra=oracion_to_ready(palabra) #normalización de la palabra

  for i,char in enumerate(palabra):
    index=letter_index(char)
    tensor_final[i,0,index]=1 #Coloco el 1 para la notación OHE

  return tensor_final

## Pruebas del 'word_to_tensor' que se usa en cada palabra en el dataset personalizado

In [10]:
res=word_to_tensor("café") #café

In [11]:
torch.argmax(res,dim=2)

tensor([[2],
        [0],
        [5],
        [4]])

In [12]:
chars_allowed[2],chars_allowed[0],chars_allowed[5],chars_allowed[4]

('c', 'a', 'f', 'e')

# Creación del dataset personalizado

In [13]:
from io import open
import glob
import os
import torch
from torch.utils.data import Dataset

In [14]:
#Dataset personalizado
class NamesDataset(Dataset):
  #Constructor
  def __init__(self,datadir):
    self.datadir=datadir

    self.data=[]
    self.labels=[]

    self.labels_total=set()

    text_archivos=glob.glob(os.path.join(datadir,"*.txt")) #Archivos que terminen en .txt dentro del path indicado

    #Bucle para recorrer cada uno de los archivos
    for archivo in text_archivos:
      palabras=open(archivo).read().strip().split('\n') #Listas de nombres (palabras)

      label=os.path.splitext(os.path.basename(archivo))[0] #Obtengo el label desde el nombre del archivo
      self.labels_total.add(label)

      for palabra in palabras:
        self.data.append(word_to_tensor(palabra))
        self.labels.append(label)

  #Len
  def __len__(self):
    return len(self.data)


  #Get item
  def __getitem__(self,idx):
    data_idx=self.data[idx]
    label_idx=self.labels[idx]

    return data_idx,label_idx

In [15]:
data_total=NamesDataset("data/names")

# Labels numéricas de las clases

In [16]:
class_dict={label:index for index,label in enumerate(data_total.labels_total)}

In [17]:
labels=torch.tensor(range(0,len(data_total.labels_total)),dtype=torch.long)

## Pruebitas del dataset class personalizado

In [19]:
len(data_total),len(data_total.labels_total)

(20074, 18)

In [20]:
data_total[1] #Vector OHE procesado

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0.]],
 
         [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [21]:
print(torch.argmax(data_total.data[1],dim=2))

tensor([[26],
        [ 1],
        [ 0],
        [ 4],
        [21]])


In [23]:
#Nombre
chars_allowed[26],chars_allowed[1],chars_allowed[0],chars_allowed[4],chars_allowed[21]

('A', 'b', 'a', 'e', 'v')

# División de datos

In [24]:
train_set,test_set=torch.utils.data.random_split(data_total,[.85, .15],generator=torch.Generator(device=device).manual_seed(2024))

In [25]:
print("Len train_set:",len(train_set), "Len test set:",len(test_set))

Len train_set: 17063 Len test set: 3011


# Creación de la red

In [26]:
class CharRNN(torch.nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(CharRNN,self).__init__()

    self.rnn=torch.nn.RNN(input_size,hidden_size)
    self.h_s=torch.nn.Linear(hidden_size,output_size)
    self.lsm=torch.nn.LogSoftmax(dim=1) #convierte los logs a probabilidades

  def forward(self,x_data):
    rnn_out,hidden=self.rnn(x_data) #rnn_out (resultado de cada time step), hidden (estado oculto final)
    #print("hidden shape:",hidden.shape)
    output=self.h_s(hidden[0]) #logits sin convertir en probs (hidden[0]), ya que inic. el vector 'hidden' es de shape (1,1,128), y se quiere (1,128) para la capa fully connected)
    output=self.lsm(output) #logits convertidos en probs y aplicado su logaritmo de cada prob

    return output #rnn_out,hidden,output

In [27]:
rnn_modelo=CharRNN(len_chars,128,len(data_total.labels_total))

In [28]:
rnn_modelo

CharRNN(
  (rnn): RNN(58, 128)
  (h_s): Linear(in_features=128, out_features=18, bias=True)
  (lsm): LogSoftmax(dim=1)
)

# Entrenamiento

In [29]:
import random

In [30]:
def train(rnn,training_data,n_epoch=10,n_batch_size=64,learning_rate=0.2,criterion=torch.nn.NLLLoss()):
  optimizer=torch.optim.Adam(rnn.parameters(),0.005) #optimizer

  loss_total=[]

  #Entrenamiento por dicha cantidad de épocas(igual debe haber un bucle interno para iterar sobre los batches)
  for epoca in range(n_epoch):

    loss_epoca=0

    num_batches=len(training_data)//n_batch_size

    idxs_batch_sample=list(range(len(training_data))) #lista de índices de cada sample del batch
    batches=np.array_split(idxs_batch_sample,num_batches) #separo en una lista de índices (cada una de tamaño del num_batches)

    #print("batches",batches)
    #print(f"num_batches: {num_batches}")

    #Itero sobre cada batch (a su vez estos van a tener los índices de cada palabras, así que también se iterará sobre cada índice de la palabra para pasarle a la RNN)
    for idx_batch,batch in enumerate(batches):
      loss_batch=0 #loss para calcular el gradiente

      #print(f"idx_batch: {idx_batch}, len batch: {len(batch)}")
      for sample in batch: #Itero sobre cada muestra del batch

        tensor_word_data,label_data=training_data[sample]

        label=labels[class_dict[label_data]] #label correcta
        y_pred=rnn(tensor_word_data) #predicción

        loss=criterion(y_pred,label.unsqueeze(0))

        loss_batch+=loss
        #print("loss_batch:",loss_batch)
        #print("tensor_word_data shape:",tensor_word_data.shape)
        #print("label_data:",label_data,class_dict[label_data],label,label.unsqueeze(0).shape) #class_dict


      optimizer.zero_grad() #Setteo los gradientes a 0 antes de calcular los gradientes de los pesos mediante la loss_batch

      loss_batch.backward() #Calculo los gradientes de los pesos con la pérdida del batch

      optimizer.step() #Actualizo los pesos de la red

      loss_epoca+=loss_batch.item()/len(batch) #Promedio del loss de cada batch se suma al loss de esa época



    #Loss promedio en cada época
    loss_total.append(loss_epoca/len(batches))

    print(f"Época {epoca}, Loss: {loss_epoca/len(batches)}")

  return loss_total

In [31]:
loss_final=train(rnn_modelo,train_set)

Época 0, Loss: 1.2199156235862563
Época 1, Loss: 0.9265171012914961
Época 2, Loss: 0.8386138929322388
Época 3, Loss: 0.7645445569479238
Época 4, Loss: 0.7211768871980158
Época 5, Loss: 0.6841448155828057
Época 6, Loss: 0.644633860119062
Época 7, Loss: 0.6286840989935805
Época 8, Loss: 0.6137491022880329
Época 9, Loss: 0.5802114986051294


# Pruebas de la red
Pruebas de clasificación de idioma dado un nombre

In [32]:
plb=word_to_tensor(palabra="Ramirez")

In [33]:
plb.shape

torch.Size([7, 1, 58])

In [45]:
#Clasificación
with torch.no_grad():
  pred=rnn_modelo(plb)
  clase_pred=torch.argmax(pred,dim=1)

print(f"Clase predicha (tensor): {clase_pred}. Diccionario: {class_dict}")

Clase predicha (tensor): tensor([7]). Diccionario: {'English': 0, 'Vietnamese': 1, 'Polish': 2, 'Czech': 3, 'Scottish': 4, 'Japanese': 5, 'Irish': 6, 'Spanish': 7, 'Greek': 8, 'Korean': 9, 'German': 10, 'Portuguese': 11, 'Arabic': 12, 'Chinese': 13, 'Italian': 14, 'Dutch': 15, 'French': 16, 'Russian': 17}


In [46]:
print(f"Clase predicha por el modelo: {list(class_dict.keys())[clase_pred.item()]}")

Clase predicha por el modelo: Spanish
